In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_community.vectorstores import Chrosma
from sentence_transformers import SentenceTransformer
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEndpoint
from dotenv import load_dotenv
import os


2025-04-01 16:14:08.842527: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743504248.930713   20217 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743504248.955469   20217 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743504249.134552   20217 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743504249.134587   20217 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743504249.134589   20217 computation_placer.cc:177] computation placer alr

In [2]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2',device="cpu")
embedding_function = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [3]:
# pdf_paths = ["pdfcoffee.com_jane-huff-ecg-workout-exercises-in-arrhythmia-interpretation-2011-pdf-free.pdf", 
#               "1garcia_tomas_b_ed_12_lead_ecg_the_art_of_interpretation.pdf"]
 
# chroma_db = Chroma(persist_directory='./home/laflamme/course/capstone/chromadb', collection_name="ECGbooks", embedding_function=embedding_function
#                    )
# for pdf in pdf_paths:
#     print(pdf)
#     loader = PyPDFLoader(pdf)
#     docs = loader.load()

#     text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

    
#     documents = text_splitter.split_documents(docs)
#     chroma_db.add_documents(documents)
    

In [4]:
db = Chroma(persist_directory='./home/laflamme/course/capstone/chromadb', collection_name="ECGbooks", embedding_function=embedding_function)


In [5]:
import pandas as pd


In [6]:
data_dir = '/home/laflamme/course/capstone/ptb-xl+'
label_file = data_dir + '/labels/mapping/12slv23ToSNOMED.csv'
labels = pd.read_csv(label_file)
labelled_data = labels['Acronym'].dropna()
# labelled_data.__sizeof__()

# len(labelled_data)

In [7]:
statement_file = data_dir + '/labels/12sl_statements.csv'
label_ecgid ={}
df = pd.read_csv(statement_file)
# statements = df['statements']

for _,row in df.iterrows():
    values = eval(row['statements']) 
    for value in values:
        label_ecgid[value] = row['ecg_id']
# len(label_ecgid)

In [8]:
def str_features(row):
    res = ""
    for index,value in row.items():
        if index=="ecg_id":
            continue
        res = res + f"The {index} value is {value}. "
    return res

In [9]:
feature_file = data_dir + '/features/12sl_features.csv'
label_ecg =[]
df = pd.read_csv(feature_file)
for _,row in df.iterrows():
    ecg_id = row['ecg_id']
    row.drop(columns=['ecg_id'],axis = 1)
    for label,id in label_ecgid.items():
        if(ecg_id==id):
            # label_ecg.append(f"The following case looks like ")
            label_ecg.append(str_features(row)+f"The diagnosis is {label}")
            
            # label_ecg[label] = row.drop(['ecg_id'])
# label_ecg.__sizeof__()

In [10]:
# db = Chroma(persist_directory='./chromadb')
# db._collection_name

In [11]:
load_dotenv()

True

In [12]:
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

# Load a text-generation pipeline from Hugging Face
hf_pipeline = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",torch_dtype="auto",device_map="cpu",max_new_tokens=256)

# Wrap the pipeline in a LangChain-compatible LLM
llm = HuggingFacePipeline(pipeline=hf_pipeline)


Device set to use cpu
/tmp/ipykernel_20217/2649650793.py:8: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=hf_pipeline)


In [13]:
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain



In [14]:
"""
Answer ONLY using the context below. If unsure, say "I don't know".

CONTEXT:
{context}

QUESTION: 
{input}

ANSWER (no markdown):
"""

'\nAnswer ONLY using the context below. If unsure, say "I don\'t know".\n\nCONTEXT:\n{context}\n\nQUESTION: \n{input}\n\nANSWER (no markdown):\n'

In [15]:
prompt = ChatPromptTemplate.from_template("""
You are a medical AI assistant specializing in ECG analysis. Your task is to examine the retrieved ECG features and determine if they indicate any abnormalities.  

### **Instructions:**  
- First, carefully review the provided ECG features in the context.  
- If the features suggest a heart condition, specify the most likely disease.  
- If you are uncertain, respond with: "The ECG features do not strongly indicate a known disease."  
- If no clear conclusion can be drawn, say: "I don't know."  

### **RETRIEVED ECG FEATURES:**  
{context}  

### **QUESTION:**  
The following ECG features were observed:  
{input}  

Do these features indicate heart arrhythmias? If so, specify the most likely disease.  

### **ANSWER (no markdown):**  
""")


In [16]:
from langchain.text_splitter import TokenTextSplitter
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

# Make sure it's splitting correctly
text_splitter = TokenTextSplitter(
    chunk_size=512,  # Limit chunk size
    chunk_overlap=50,
    length_function=lambda x: len(tokenizer.encode(x, add_special_tokens=False))  # Exclude special tokens
)

document_chain = create_stuff_documents_chain(llm, prompt)
retriever = db.as_retriever(
    search_kwargs={'k': 5},  # Reduce retrieved documents
    text_splitter=text_splitter
)
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [17]:
labels={}
df = pd.read_csv(statement_file)
# statements = df['statements']
for _,row in df.iterrows():
    values = eval(row['statements']) 
    if len(values)!=0:
        labels[row['ecg_id']] = values[0]

In [18]:

label_features ={}
df = pd.read_csv(feature_file)
for _,row in df.iterrows():
    ecg_id = row['ecg_id']
    row.drop(columns=['ecg_id'],axis = 1)
    # print(ecg_id)
    for id,label in labels.items():
        # print(f"bruh {id}")
        if(ecg_id==id):
            # label_ecg.append(f"The following case looks like ")
            label_features[label] = str_features(row)
            # print(len(label_features))
    if len(label_features)>10:
        break
# label_features


In [ ]:
MAX_TOKENS = 2048

def truncate_text(text, tokenizer, max_tokens=MAX_TOKENS):
    tokens = tokenizer.encode(text, add_special_tokens=False)
    truncated_tokens = tokens[:max_tokens]
    return tokenizer.decode(truncated_tokens)

# Before passing input to the LLM
processed_text = truncate_text(label_features["NSR"], tokenizer)
response = retrieval_chain.invoke({"input": processed_text})


Token indices sequence length is longer than the specified maximum sequence length for this model (12969 > 2048). Running this sequence through the model will result in indexing errors


Token indices sequence length is longer than the specified maximum sequence length for this model (2858 > 2048). Running this sequence through the model will result in indexing errors
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (2048). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


In [20]:
response

{'input': 'The P_Area_I value is 0.116. The P_PeakTime_I value is 76.0. The Q_Area_I value is 0.0. The Q_PeakTime_I value is 0.0. The R_Area_I value is 0.288. The R_PeakTime_I value is 34.0. The S_Area_I value is 0.456. The S_PeakTime_I value is 62.0. The QRS_Balance_I value is -25.0. The T_Area_I value is 1.886. The T_PeakTime_I value is 74.0. The QRS_Area_I value is -0.167. The P_Area_II value is 0.38. The P_PeakTime_II value is 52.0. The Q_Area_II value is 0.051. The Q_PeakTime_II value is 8.0. The R_Area_II value is 1.917. The R_PeakTime_II value is 42.0. The S_Area_II value is 0.147. The S_PeakTime_II value is 74.0. The QRS_Balance_II value is 1758.0. The T_Area_II value is 3.581. The T_PeakTime_II value is 70.0. The QRS_Area_II value is 1.73. The P_Area_V1 value is 0.103. The P_PeakTime_V1 value is 44.0. The Q_Area_V1 value is 0.0. The Q_PeakTime_V1 value is 0.0. The R_Area_V1 value is 0.483. The R_PeakTime_V1 value is 20.0. The S_Area_V1 value is 0.72. The S_PeakTime_V1 value is

In [31]:
response['answer']

'Human: \nYou are a medical AI assistant specializing in ECG analysis. Your task is to examine the retrieved ECG features and determine if they indicate any abnormalities.  \n\n### **Instructions:**  \n- First, carefully review the provided ECG features in the context.  \n- If the features suggest a heart condition, specify the most likely disease.  \n- If you are uncertain, respond with: "The ECG features do not strongly indicate a known disease."  \n- If no clear conclusion can be drawn, say: "I don\'t know."  \n\n### **RETRIEVED ECG FEATURES:**  \nPR interval: ___________________________ QRS complex: __________________\nStrip 5-3. Rhythm: _____________________________ Rate: ________________________  P wave: __________________\n PR interval: ___________________________ QRS complex: __________________\nRhythm strip practice: Analyzing rhythm strips\nAnalyze the following rhythm strips using the ﬁ  ve-step process discussed in this chapter. Check your answers with the \n answer key in 

In [21]:
# test_cases = []
# responses = []
# for label, features in label_features.items():
#     query = f"Following are the ECG features :\n{features}\n Can they be associated with heart arrhythmias? If yes, specify the proper disease name."
#     responses.append(retrieval_chain.invoke({"input":f"{features}"}))

In [22]:
# from transformers import AutoTokenizer

# # Initialize tokenizer
# tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0",device = "cpu")

In [23]:
# def chunk_text(text, tokenizer, chunk_size=1024, overlap=256):
#     """Splits text into overlapping tokenized chunks."""
#     tokens = tokenizer.encode(text, truncation=False)  # Get token IDs
#     chunks = []
    
#     for i in range(0, len(tokens), chunk_size - overlap):
#         chunk = tokens[i: i + chunk_size]
#         chunks.append(tokenizer.decode(chunk))  # Convert back to text
#     return chunks

In [24]:
# from transformers import AutoTokenizer

# # Initialize tokenizer
# tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

# def chunk_text(text, tokenizer, chunk_size=1024, overlap=256):
#     """Splits text into overlapping tokenized chunks."""
#     tokens = tokenizer.encode(text, truncation=False)  # Convert text to token IDs
#     chunks = []
    
#     for i in range(0, len(tokens), chunk_size - overlap):
#         chunk = tokens[i: i + chunk_size]
#         chunks.append(tokenizer.decode(chunk))  # Convert back to text
    
#     return chunks

# test_cases = []
# for label, features in label_features.items():
#     query = f"Following are the ECG features :\n{features}\n Can they be associated with heart arrhythmias? If yes, specify the proper disease name."
#     retrieved_docs = retriever.invoke(query)
#     retrieved_contexts = [doc.page_content for doc in retrieved_docs]
    
#     print(f" Retrieved {len(retrieved_contexts)} Documents:")  
#     for i, ctx in enumerate(retrieved_contexts[:3]):  
#         print(f"   {i+1}. {ctx[:200]}...")  

#     # Combine retrieved contexts and chunk them
#     full_context = "\n".join(retrieved_contexts)
#     context_chunks = chunk_text(full_context, tokenizer)
    
#     generated_responses = []
#     for chunk in context_chunks:
#         generated_answer = retrieval_chain.invoke({
#             "context": chunk,
#             "input": query
#         })

#         # Extract response text safely
#         if isinstance(generated_answer, dict):
#             generated_text = generated_answer.get("output", "")  # Adjust based on key
#         else:
#             generated_text = str(generated_answer)

#         generated_responses.append(generated_text)

#     # Aggregate responses into a final answer
#     final_answer = " ".join(generated_responses)  

#     print(f"Generated Answer: {final_answer}\n") 
#     test_cases.append({
#         "query": query,
#         "ground_truth": label,
#         "retrieved_context": retrieved_contexts,
#         "generated_answer": final_answer
#     })

# test_cases = []
# for label, features in label_features.items():
#     query = f"Following are the ECG features :\n{features}\n Can they be assocaiated with heart arrythmias. If yes, specify the proper disease name"
#     retrieved_docs = retriever.invoke(query)
#     retrieved_contexts = [doc.page_content for doc in retrieved_docs]

#     print(f" Retrieved {len(retrieved_contexts)} Documents:")  
#     full_context = "\n".join(retrieved_contexts)
#     context_chunks = chunk_text(full_context, tokenizer)
#     # generated_answer = retrieval_chain.invoke({"input":query})
#     generated_responses = []
#     for chunk in context_chunks:
#         generated_answer = retrieval_chain.invoke({
#             "context": chunk,
#             "input": query
#         })
#         generated_responses.append(generated_answer)

#     # Aggregate responses (concatenation or some logic)
#     # final_answer = " ".join(generated_responses)  
    
#     for i, ctx in enumerate(retrieved_contexts[:3]):  
#         print(f"   {i+1}. {ctx[:200]}...")  
#     print(f"Generated Answer: {generated_answer}\n") 
#     test_cases.append({
#         "query" : query,
#         "ground_truth":label,
#         "retrieved_context": retrieved_contexts,
#         "generated_answer": final_answer
#     })

In [25]:
import csv
csv_file_path = "/home/laflamme/course/capstone/implementation/fewshot/test_cases.csv"

# Open and read the CSV file
with open(csv_file_path, mode='r') as file:
    # Use DictReader to read rows as dictionaries
    reader = csv.DictReader(file)
    
    # Extract data into a list of dictionaries
    test_cases = [row for row in reader]

# Print the extracted data
# for row in data:
#     print(row)
# with open(csv_file_path,"w",newline="") as file:
#     writer = csv.DictWriter(file, fieldnames=test_cases[0].keys())
#     writer.writeheader()
#     writer.writerows(test_cases)

In [26]:
# labels = generate_labels(statement_file=statement_file)
# label_features = generate_label_features(feature_file=feature_file,labels=labels)
# test_cases = create_test_cases(label_features,retrieval_chain,retriever)
# print(1)


In [27]:
# llm_eval = HuggingFaceEndpoint(
#     repo_id="google/flan-t5-base",
#     task="text-generation",
#     huggingfacehub_api_token=""

# )
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

# Load a text-generation pipeline from Hugging Face
hf_pipeline = pipeline("text2text-generation", model="google/flan-t5-base",torch_dtype="auto",max_new_tokens=256)

# Wrap the pipeline in a LangChain-compatible LLM
llm_eval = HuggingFacePipeline(pipeline=hf_pipeline)


Device set to use cuda:0


In [28]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [29]:
from ragas import EvaluationDataset, SingleTurnSample
from ragas.metrics import Faithfulness,  AnswerRelevancy, context_precision, context_recall
from ragas.evaluation import evaluate

In [30]:

import importlib
import evaluation_utils2 as ev2

importlib.reload(ev2)
n = len(test_cases)
scores = 0
for case in test_cases[:1]:
    print(type(case["retrieved_context"]))
    scores = scores + ev2.llm_as_a_judge(llm= llm_eval,query=case["query"],retrieved_docs=case["retrieved_context"],reference_answer=case["ground_truth"])
print(scores/n)

/home/laflamme/course/capstone/implementation/fewshot/evaluation_utils2.py:27: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(prompt)
Token indices sequence length is longer than the specified maximum sequence length for this model (11697 > 512). Running this sequence through the model will result in indexing errors


<class 'str'>


OutOfMemoryError: CUDA out of memory. Tried to allocate 6.12 GiB. GPU 0 has a total capacity of 3.63 GiB of which 2.25 GiB is free. Including non-PyTorch memory, this process has 1.33 GiB memory in use. Of the allocated memory 1.19 GiB is allocated by PyTorch, and 82.01 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:

import importlib
import evaluation_utils as ev

importlib.reload(ev)
results = ev.RAGA_Evaluator(test_cases=[{'query': 'What is the capital of France?',
                                            'ground_truth': 'Paris',
                                            'retrieved_context': 'France is a country in Europe, and its capital is Paris.',
                                            'generated_answer': 'Paris'}],
                            llm=llm_eval, embeddings=embedding_function)



In [ ]:
type(results)


In [ ]:
results

In [ ]:
# Convert results to DataFrame
df1 = results.to_pandas()

# Iterate through rows
for _, row in df1.iterrows():
    print(row)

In [ ]:

print("\n🔹 Evaluation Results:")

In [ ]:

# response = retrieval_chain.invoke({
#     "context": "\n".join(label_ecg),
#     "input": f"Following are the ECG features :\n{label_ecg[0]}\n Can they be assocaiated with heart arrythmias. If yes, specify the proper disease name"
# })
# print(response['input'])

# print(response['answer'])

In [ ]:
# document_chain = create_stuff_documents_chain(llm, prompt)
# retriever = db.as_retriever(   search_type="mmr",
#     search_kwargs={'k': 5, 'fetch_k': 50})
# retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:

# response = retrieval_chain.invoke({
#     "context": "\n".join(label_ecg),
#     "input": f"Following are the ECG features :\n{label_ecg[0]}\n Can they be assocaiated with heart arrythmias. If yes, specify the proper disease name"
# })
# print(response['input'])

# print(response['answer'])

In [ ]:
# document_chain = create_stuff_documents_chain(llm, prompt)
# retriever = db.as_retriever(    search_type="similarity_score_threshold",
#     search_kwargs={'score_threshold': 0.5})
# retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:

# response = retrieval_chain.invoke({
#     "context": "\n".join(label_ecg),
#     "input": f"Following are the ECG features :\n{label_ecg[0]}\n Can they be assocaiated with heart arrythmias. If yes, specify the proper disease name"
# })
# print(response['input'])

# print(response['answer'])